IMPORTS E PARÂMETROS

In [1]:
import tabula as tb
import pandas as pd
import numpy  as np

FUNÇÕES

In [2]:
def get_normalized_java_texts():
    normalized_java_texts = {
        "1/3 Férias Total Férias": "13_ferias total_ferias", 
        "1/3 Férias"            : "13_ferias", 
        "Total Férias"          : "total_ferias", 
        "13o Salário"           : "13_salario", 
        "TOTAL"                 : "total", 
        "Apropriação"           : "apropriacao", 
        "INSS 20,00%"           : "inss", 
        ".*Ac. Trab. .*"        : "ac_trab", 
        "FGTS 8%"               : "fgts", 
        "SP-PREVCOM"            : "spprevcom", 
        "Total de Encargos"     : "total_encargos", 
        "Férias em pecúnia"     : "ferias_pecunia", 
        "SPPREV"                : "spprev", 
        "Férias"                : "ferias", 
        "\."                    : "",
        ","                     : ".",
        "APROPRIAÇÃO MENSAL"    : "apropriado",
        "REALIZADO MENSAL"      : "realizado",
        "SALDO FINAL MENSAL"    : "saldo",
        "REVERSÃO / REVISÃO DAS APROPRIAÇÕES" : "revertido",
        "Total"                 : "total_ferias",
    }
    return normalized_java_texts

def get_table_types():
    table_types =  [
        "CLT", 
        "AUTÁRQUICO", 
    ]
    return table_types

def get_ignored_table_types():
    ignored_table_types = [
        "TOTAIS", 
    ]
    return ignored_table_types

def get_renamed_columns():
    renamed_collumns = {
        "Unnamed: 0": "tipo", 
        "Unnamed: 1":"Col1", 
        "Col1":"Col2", 
        "Col2":"Col3", 
        "Unnamed: 2":"Col4", 
        "Unnamed: 3":"Col5",
    }
    return renamed_collumns

def get_reordered_columns():
    reordered_columns = [
    "Unnamed: 0", 
    "Unnamed: 1", 
    "Col1", 
    "Col2", 
    "Unnamed: 2", 
    "Unnamed: 3", 
    ]
    return reordered_columns

def get_internal_tables():
    internal_tables = [
        "apropriado", 
        "realizado", 
        "saldo", 
        "revertido", 
    ]
    return internal_tables

def normalize_java_text(tables:list) -> list:
    for index, table in enumerate(tables, start=1):
        tables[index-1] = table.replace(get_normalized_java_texts(), inplace=False, regex=True)
    return tables

def get_table_type(table:list) -> str:
    if not any(ignore_type in column for column in table for ignore_type in get_ignored_table_types()):
        for type in get_table_types():
            if type in table.columns:
                return type
    return None

def split_column(table:list, type:str, col1="Col1", col2="Col2") -> pd.DataFrame:
    df = pd.DataFrame(columns=[col1, col2])
    for value in table[type]:
        values = value.split()
        df.loc[len(df)] = [values[0], values[1] if len(values) > 1 else None]
    return df

def join_columns(table:list, df:pd.DataFrame, type:str) -> pd.DataFrame:
    table.drop(columns=[type], inplace=True)
    table = pd.concat([table, df], axis=1)
    return table

def format_data_frame(df:pd.DataFrame, type:str)->pd.DataFrame:
    df = df.reindex(columns=get_reordered_columns())
    colunas_renomeadas = get_renamed_columns()
    colunas_renomeadas["Unnamed: 0"] = type
    df = df.rename(columns=colunas_renomeadas)
    return df

def get_line_positions(df:pd.DataFrame, col:str="Col2") -> list:
    positions = []
    for index, line in enumerate(df[col]):
        if line in get_internal_tables():
            positions.append(index)
    positions.append(len(df))
    return positions

def split_internal_dataframe(df_main:pd.DataFrame, pos:list):
    results = []
    for i in range(len(pos) - 1):
            df_new  = df_main[pos[i] : pos[i+1]]
            type_df = df_new.iloc[0,2]
            df_new  = df_new.rename(columns=df_new.iloc[1].replace(np.nan, ""))[2:]
            df_new.reset_index(drop=True, inplace=True)
            results.append((type_df, df_new))
    return results

def get_dataframes(df_main:pd.DataFrame) -> list:
    type = get_table_type(df_main)
    if type is not None:
        df_main = join_columns(df_main, split_column(df_main, type), type)
        df_main = format_data_frame(df_main, type)
        return split_internal_dataframe(df_main, get_line_positions(df_main))
    else:
        return []

def get_tables_from(file_path:str)->dict:
    output = {}
    tables = normalize_java_text(tb.read_pdf(file_path, pages="all"))
    for idx, table in enumerate(tables, start=1):
        results = get_dataframes(table)
        table_results = [] 
        for table_type, sub_tables in results:
            table_results.append((table_type, sub_tables))
        output[idx] = table_results
    return output

EXECUÇÃO PRINCIPAL

In [3]:
output = get_tables_from("RHPP05LA - GERAL.pdf")

for key, value in output.items():
    for tipo_tabela, tabela in value:
            display(tipo_tabela)
            display(tabela)

abr 17, 2024 4:20:53 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTÊNCIA: Using fallback font LiberationSans for base font Symbol
abr 17, 2024 4:20:53 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
ADVERTÊNCIA: Using fallback font LiberationSans for base font ZapfDingbats


'apropriado'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,60133.58,20044.36,80177.94,60133.58,140311.52
1,inss,12026.48,4008.66,16035.14,9991.98,26027.12
2,ac_trab,1074.44,357.95,1432.39,892.65,2325.04
3,fgts,4810.46,1603.31,6413.77,3996.63,10410.40
4,spprevcom,0.00,0.00,0.00,718.24,718.24
5,total_encargos,17911.38,5969.92,23881.30,15599.50,39480.80
6,total,78044.96,26014.28,104059.24,75733.08,179792.32


'realizado'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,80594.28,28188.65,108782.93,0.00,108782.93
1,ferias_pecunia,15287.07,0.00,15287.07,0.00,15287.07
2,total_ferias,95881.35,28188.65,124070.00,0.00,124070.00
3,inss,19176.23,5637.66,24813.89,0.00,24813.89
4,ac_trab,1713.52,503.72,2217.24,0.00,2217.24
5,fgts,7670.44,2255.00,9925.44,0.00,9925.44
6,spprevcom,0.00,0.00,0.00,0.00,0.00
7,total_encargos,28560.19,8396.38,36956.57,0.00,36956.57
8,total,124441.54,36585.03,161026.57,0.00,161026.57


...

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,146256.84,49705.68,195962.52,0.00,195962.52
1,spprevcom,0.00,0.00,0.00,0.00,0.00
2,total,146256.84,49705.68,195962.52,0.00,195962.52
3,spprev,0.00,0.00,0.00,0.00,0.00


'saldo'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,143681.76,47781.35,191463.11,261645.95,453109.06
1,spprevcom,0.00,0.00,0.00,164.54,164.54
2,total,143681.76,47781.35,191463.11,261810.49,453273.60
3,spprev,0.00,0.00,0.00,62713.46,62713.46


'revertido'

,,ferias,13_ferias,total_ferias,13_salario,total
0,apropriacao,93989.58,31441.96,125431.54,0.00,125431.54
1,spprevcom,0.00,0.00,0.00,0.00,0.00
2,total,93989.58,31441.96,125431.54,0.00,125431.54
3,spprev,0.00,0.00,0.00,0.00,0.00
